**เชื่อม Google Drive + ติดตั้ง ComfyUI** ✈

---



In [ ]:
#@title เชื่อม Google Drive + ติดตั้ง ComfyUI
from google.colab import drive
from IPython.display import clear_output
import sys
import os

google_drive = True #@param {type:"boolean"}

# Mount Google Drive
if google_drive:
    !echo -e "\e[1;32mกำลังเชื่อมต่อกับ google drive...\e[0m"
    drive.mount('/content/drive')
    !echo -e "\e[1;32mเชื่อมต่อกับ google drive เสร็จสิ้น\e[0m"

drive_type = 'My Drive' #@param ["My Drive", "Shared drives"]

if drive_type == 'Shared drives':
    shared_drive_name = 'AI' #@param {type:"string"}

#foder path
folder_path = 'ComfyUI' #@param {type:"string"}

#remove Space
drive_type = drive_type.replace(' ', '')

# Set path
if google_drive:

    #check if shared drive
    if drive_type == 'Shareddrives':
        path = '/content/drive/' + drive_type + '/' + shared_drive_name + '/' + folder_path + '/'

        if not os.path.exists('/content/drive/' + drive_type + '/' + shared_drive_name):
            !echo -e "\e[1;31mไม่พบ Shared Drive ที่ระบุ โปรดลองใหม่อีกครั้ง!\e[0m"
            !echo -e "\e[1;31mหากต้องการเปลี่ยน account โปรดหยุด runtime แล้วเปิด runttime ใหม่อีกครั้ง!\e[0m"
            sys.exit()
    else:
        path = '/content/drive/' + drive_type + '/' + folder_path + '/'
        if not os.path.exists('/content/drive/' + drive_type):
            !echo -e "\e[1;31mระบบเชื่อมต่อ Google Drive ไม่สำเสร็จ โปรดลองใหม่อีกครั้ง!\e[0m"
            !echo -e "\e[1;31mหากต้องการเปลี่ยน account โปรดหยุด runtime แล้วเปิด runttime ใหม่อีกครั้ง!\e[0m"
            sys.exit()

    #check if path exists and create folder if not
    if not os.path.exists(path):
        os.makedirs(path)
else:
    path = '/content/'

%cd $path
#check file in path folder if yes no git clone
if not os.path.exists(path + 'main.py'):
    !echo -e "\e[1;32mกำลังติดตั้ง...\e[0m"
    !git clone https://github.com/comfyanonymous/ComfyUI.git
    !echo -e "\e[1;32mติดตั้งสำเร็จ\e[0m"
else:
    update_comfyui = True #@param {type:"boolean"}
    if update_comfyui:
        !echo -e "\e[1;32mกำลังอัปเดต...\e[0m"
        !git -C path pull
        !echo -e "\e[1;32mอัปเดตเสร็จสิ้น\e[0m"

#path
path = path + 'ComfyUI/'

%cd $path

#install requirements
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 xformers
!pip install -r requirements.txt

#clear output
clear_output()
!echo -e "\e[1;32mการทำงานเสร็จสมบูรณ์!\e[0m"

# **ดาวน์โหลด Model , Checkpoint , Lora และ อื่นๆ**


---


In [ ]:
#@title ดาวน์โหลด Model , Checkpoint , Lora และ อื่นๆ
#@markdown ดาวน์โหลดผ่าน URL
import os.path as Path

model_url = "https://civitai.com/api/download/models/105924" #@param {type:"string"}
model_type = "checkpoint" #@param ["checkpoint", "lora" , "controlnet" , "embeding" , "vae"] {allow-input: true}
model_name = "CetusMix_WhaleFall2" #@param {type:"string"}
model_extension = "safetensors" # @param ["safetensors", "pth", "ckpt"] {allow-input: true}
#@markdown <font color="#ff4d4d">กรุณากรอกข้อมูลในช่องให้เรียบร้อย เพื่อความสะดวกต่อการใช้งานโมเดลที่คุณโหลดในอนาคต</font>

if model_name == "":
    #file name from url
    file_name = model_url.split('/')[-1] + '.' + model_extension
else:
    file_name = model_name + '.' + model_extension

#file path
model_type += "s" if model_type != "controlnet" else ""
file_path = path + '/models/' + model_type + '/' +  file_name
#check if file exists
if not Path.exists(file_path):
    !echo -e "\e[1;32mกำลังดาวน์โหลดไฟล์"  {file_name}  "...\e[0m"
    !wget -O - {model_url} > {file_path}
    !echo -e "\e[1;32mดาวน์โหลดไฟล์"  {file_name}  "เสร็จสมบูรณ์!\e[0m"
else:
    !echo -e "\e[1;31ไฟล์" {file_name} "มีอยู่แล้วในระบบ!\e[0m"

## ดาวน์โหลดอย่างง่าย

In [ ]:
#@title ดาวน์โหลด SD 1.5 Model อย่างง่าย
import os.path as Path

sd = {}

#@markdown <hr><h3>Checkpoint + VAE</h3><hr>

#@markdown <font color="#ccc">Checkpoint พื้นฐานสุด <font color="#ff2b3d">(ไม่แนะนำ)</font></font>
CHECKPOINT_PRUNED_EMAONLY = False #@param {type:"boolean"}
#@markdown <font color="#ccc">VAE พื้นฐาน <font color="#03fc5e">(แนะนำ)</font></font>
VAE_FT_MSE = False #@param {type:"boolean"}




if CHECKPOINT_PRUNED_EMAONLY:
    file_name = 'v1-5-pruned-emaonly.ckpt'
    file_path = path + 'models/checkpoints/' +  file_name
    #check if file exists
    if not Path.exists(file_path):
        !echo -e "\e[1;32mกำลังดาวน์โหลด" {file_name} "...\e[0m"
        !curl -Lo {file_path} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt
        !echo -e "\e[1;32mดาวน์โหลด" {file_name} "เสร็จเรียบร้อย!\e[0m"
    else:
        !echo -e "\e[1;32m" {file_name} "มีอยู่ในระบบอยู่แล้ว!\e[0m"

if VAE_FT_MSE:
    file_name = 'vae-ft-mse-840000-ema-pruned.safetensors'
    file_path = path + 'models/vae/' +  file_name
    #check if file exists
    if not Path.exists(file_path):
        !echo -e "\e[1;32mกำลังดาวน์โหลด" {file_name} "...\e[0m"
        !curl -Lo {file_path} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
        !echo -e "\e[1;32mดาวน์โหลด" {file_name} "เสร็จเรียบร้อย!\e[0m"
    else:
        !echo -e "\e[1;32m" {file_name} "มีอยู่ในระบบอยู่แล้ว!\e[0m"




#@markdown <hr><h3>ControlNet</h3><hr>





controlnet_name = {
    'ip2p' : 'v11e_sd15',
    'shuffle' : 'v11e_sd15',
    'tile' : 'v11f1e_sd15',
    'depth' : 'v11f1p_sd15',
    'canny' : 'v11p_sd15',
    'inpaint' : 'v11p_sd15',
    'lineart_anime' : 'v11p_sd15s2',
    'lineart' : 'v11p_sd15',
    'normalbae' : 'v11p_sd15',
    'openpose' : 'v11p_sd15',
    'scribble' : 'v11p_sd15',
    'seg' : 'v11p_sd15',
    'softedge' : 'v11p_sd15',
    'mlsd' : 'v11p_sd15',
}

ControlNet_ip2p = False #@param {type:"boolean"}
ControlNet_shuffle = False #@param {type:"boolean"}
ControlNet_tile = False #@param {type:"boolean"}
ControlNet_depth = False #@param {type:"boolean"}
ControlNet_canny = False #@param {type:"boolean"}
ControlNet_inpaint = False #@param {type:"boolean"}
ControlNet_lineart = False #@param {type:"boolean"}
ControlNet_lineart_anime = False #@param {type:"boolean"}
ControlNet_normalbae = False #@param {type:"boolean"}
ControlNet_openpose = False #@param {type:"boolean"}
ControlNet_scribble = False #@param {type:"boolean"}
ControlNet_seg = False #@param {type:"boolean"}
ControlNet_softedge = False #@param {type:"boolean"}
ControlNet_mlsd = False #@param {type:"boolean"}

#sd controlnet
sd['controlnet_ip2p'] = ControlNet_ip2p
sd['controlnet_shuffle'] = ControlNet_shuffle
sd['controlnet_tile'] = ControlNet_tile
sd['controlnet_depth'] = ControlNet_depth
sd['controlnet_canny'] = ControlNet_canny
sd['controlnet_inpaint'] = ControlNet_inpaint
sd['controlnet_lineart'] = ControlNet_lineart
sd['controlnet_lineart_anime'] = ControlNet_lineart_anime
sd['controlnet_normalbae'] = ControlNet_normalbae
sd['controlnet_openpose'] = ControlNet_openpose
sd['controlnet_scribble'] = ControlNet_scribble
sd['controlnet_seg'] = ControlNet_seg
sd['controlnet_softedge'] = ControlNet_softedge
sd['controlnet_mlsd'] = ControlNet_mlsd

#check loop controlnet name and download
for name, value in sd.items():
    if value:
        file_name =  'controlnet_'+ controlnet_name[name.replace("controlnet_", "")] + '_' + name.replace("controlnet_", "") + '.pth'
        file_path = path + 'models/controlnet/' +  file_name
        #check if file exists
        if not Path.exists(file_path):
            !echo -e "\e[1;32mกำลังดาวน์โหลด" {file_name} "...\e[0m"
            !curl -Lo {file_path} https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/{file_name}
            !echo -e "\e[1;32mดาวน์โหลด" {file_name} "เสร็จเรียบร้อย!\e[0m"
        else:
            !echo -e "\e[1;32m" {file_name} "มีอยู่ในระบบอยู่แล้ว!\e[0m"

!echo -e "\e[1;32mการดาวน์โหลดเสร็จสมบูรณ์!\e[0m"

In [ ]:
#@title ดาวน์โหลด SDXL 1.0 Model อย่างง่าย
#@markdown <hr><h3>SDXL 1.0 BASE + REFINER + VAE</h3><hr>

SDXL_BASE = True #@param {type:"boolean"}
SDXL_REFINER = False #@param {type:"boolean"}
SDXL_VAE = True #@param {type:"boolean"}
SDXL_VAE_FP16 = True #@param {type:"boolean"}

sdxl = {}
sdxl['base'] = SDXL_BASE #sdxl base
sdxl['refiner'] = SDXL_REFINER #sdxl refiner
sdxl['vae'] = SDXL_VAE #sdxl vae
sdxl['vae_fp16'] = SDXL_VAE_FP16 #sdxl vae fp16

#@markdown <hr><h3>ControlNet</h3><hr>


#sdxl controlnet
sdxl_controlnet_canny = True #@param {type:"boolean"}
sdxl_controlnet_openopose = True #@param {type:"boolean"}
sdxl_controlnet_depth = True #@param {type:"boolean"}

sdxl['controlnet_canny'] = sdxl_controlnet_canny
sdxl['controlnet_openpose'] = sdxl_controlnet_openopose
sdxl['controlnet_depth'] = sdxl_controlnet_depth


sdxl_model = {}
sdxl_model['sdxl_base_1.0'] = 'https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors'
sdxl_model['sdxl_refiner_1.0'] = 'https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors'
sdxl_model['sdxl_vae_1.0'] = 'https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors'
sdxl_model['sdxl_vae_1.0_fp16'] = 'https://huggingface.co/madebyollin/sdxl-vae-fp16-fix/resolve/main/sdxl_vae.safetensors'

sdxl_model['sdxl_controlnet_canny'] = 'https://huggingface.co/diffusers/controlnet-canny-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors'
sdxl_model['sdxl_controlnet_openpose'] = 'https://huggingface.co/thibaud/controlnet-openpose-sdxl-1.0/resolve/main/diffusion_pytorch_model.bin'
sdxl_model['sdxl_controlnet_depth'] = 'https://huggingface.co/diffusers/controlnet-depth-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors'

#download model
file_path = path + 'models/' + 'checkpoints'

if SDXL_BASE:
    file_name = 'sd_xl_base_1.0_0.9vae.safetensors'
    if not Path.exists(file_path + '/' + file_name):
        !echo -e "\e[1;32mกำลังดาวน์โหลดไฟล์"  {file_name}  "\e[0m"
        !curl -Lo {file_path + '/' + file_name} {sdxl_model['sdxl_base_1.0']}
        !echo -e "\e[1;32mดาวน์โหลดไฟล์"  {file_name}  "เสร็จสมบูรณ์!\e[0m"
    else:
        !echo -e "\e[1;31mไฟล์"  {file_name}  "มีอยู่แล้ว!\e[0m"

if SDXL_REFINER:
    file_name = 'sd_xl_refiner_1.0_0.9vae.safetensors'
    if not Path.exists(file_path + '/' + file_name):
        !echo -e "\e[1;32mกำลังดาวน์โหลดไฟล์"  {file_name}  "\e[0m"
        !curl -Lo {file_path + '/' + file_name} {sdxl_model['sdxl_refiner_1.0']}
        !echo -e "\e[1;32mดาวน์โหลดไฟล์"  {file_name}  "เสร็จสมบูรณ์!\e[0m"
    else:
        !echo -e "\e[1;31mไฟล์"  {file_name}  "มีอยู่แล้ว!\e[0m"

file_path = path + 'models/' + 'vae'

if SDXL_VAE:
    file_name = 'sdxl_vae.safetensors'
    if not Path.exists(file_path + '/' + file_name):
        !echo -e "\e[1;32mกำลังดาวน์โหลดไฟล์"  {file_name}  "\e[0m"
        !curl -Lo {file_path + '/' + file_name} {sdxl_model['sdxl_vae_1.0']}
        !echo -e "\e[1;32mดาวน์โหลดไฟล์"  {file_name}  "เสร็จสมบูรณ์!\e[0m"
    else:
        !echo -e "\e[1;31mไฟล์"  {file_name}  "มีอยู่แล้ว!\e[0m"


if SDXL_VAE_FP16:
    file_name = 'sdxl_vae_fp16.safetensors'
    if not Path.exists(file_path + '/' + file_name):
        !echo -e "\e[1;32mกำลังดาวน์โหลดไฟล์"  {file_name}  "\e[0m"
        !curl -Lo {file_path + '/' + file_name} {sdxl_model['sdxl_vae_1.0_fp16']}
        !echo -e "\e[1;32mดาวน์โหลดไฟล์"  {file_name}  "เสร็จสมบูรณ์!\e[0m"
    else:
        !echo -e "\e[1;31mไฟล์"  {file_name}  "มีอยู่แล้ว!\e[0m"

file_path = path + 'models/' + 'controlnet'

if sdxl_controlnet_canny:
    file_name = 'sdxl_controlnet_canny.safetensors'
    if not Path.exists(file_path + '/' + file_name):
        !echo -e "\e[1;32mกำลังดาวน์โหลดไฟล์"  {file_name}  "\e[0m"
        !curl -Lo {file_path + '/' + file_name} {sdxl_model['sdxl_controlnet_canny']}
        !echo -e "\e[1;32mดาวน์โหลดไฟล์"  {file_name}  "เสร็จสมบูรณ์!\e[0m"
    else:
        !echo -e "\e[1;31mไฟล์"  {file_name}  "มีอยู่แล้ว!\e[0m"

if sdxl_controlnet_openopose:
    file_name = 'sdxl_controlnet_openpose.bin'
    if not Path.exists(file_path + '/' + file_name):
        !echo -e "\e[1;32mกำลังดาวน์โหลดไฟล์"  {file_name}  "\e[0m"
        !curl -Lo {file_path + '/' + file_name} {sdxl_model['sdxl_controlnet_openpose']}
        !echo -e "\e[1;32mดาวน์โหลดไฟล์"  {file_name}  "เสร็จสมบูรณ์!\e[0m"
    else:
        !echo -e "\e[1;31mไฟล์"  {file_name}  "มีอยู่แล้ว!\e[0m"


if sdxl_controlnet_depth:
    file_name = 'sdxl_controlnet_depth.safetensors'
    if not Path.exists(file_path + '/' + file_name):
        !echo -e "\e[1;32mกำลังดาวน์โหลดไฟล์"  {file_name}  "\e[0m"
        !curl -Lo {file_path + '/' + file_name} {sdxl_model['sdxl_controlnet_depth']}
        !echo -e "\e[1;32mดาวน์โหลดไฟล์"  {file_name}  "เสร็จสมบูรณ์!\e[0m"
    else:
        !echo -e "\e[1;31mไฟล์"  {file_name}  "มีอยู่แล้ว!\e[0m"



# จัดการ Custom Node

In [ ]:
#@title ดาวน์โหลด Custom Node
clone_node_project = 'https://github.com/ltdrdata/ComfyUI-Manager.git' #@param {type:"string"}
%cd {path}/custom_nodes
!echo -e "\e[1;32mกำลังติดตั้งส่วนเสริม...\e[0m"
!git clone {clone_node_project}
!echo -e "\e[1;32mการติดตั้งเสร็จสมบูรณ์!\e[0m"

In [ ]:
#@title อัปเดต Custom Node
import os
%cd {path}/custom_nodes
Update_All_Nodes = True #@param {type: "boolean"}
#@markdown <hr>
Update_Node = False #@param {type: "boolean"}
Force_Pull = False #@param {type: "boolean"}
Node_folder = 'ComfyUI-Manager' #@param {type: "string"}

if Update_All_Nodes:
    #check and delete file name example_node.py.example
    if os.path.exists(os.path.join(path,'custom_nodes','example_node.py.example')):
        os.remove(os.path.join(path,'custom_nodes','example_node.py.example'))
    for x in os.listdir():
        x = os.path.join(path + '/custom_nodes',x)
        %cd {x}
        !git pull
        #get last folder name
        folder_name = os.path.basename(os.path.normpath(x))
        !echo -e "\e[1;32mอัปเดต " { folder_name} "เสร็จสมบูรณ์!\e[0m"
elif Update_Node:
    #check if folder node_folder exists
    p = os.path.join(path,'custom_nodes')
    if not os.path.exists(os.path.join(p,Node_folder)):
        !echo -e "\e[31mหาโฟเดอร์" {Node_folder} "ไม่เจอ\e[0m"
    else:
        if Force_Pull:
            %cd $Node_folder
            !git fetch --all
            !git reset --hard origin
        !git pull
        !echo -e "\e[1;32mการอัปเดตเสร็จสมบูรณ์!\e[0m"
else:
    !echo -e "\e[31mกรุณาเลือกการอัปเดต\e[0m"

# เปิดใช้งาน ComfyUI

In [ ]:
#@title เปิดใช้งานด้วย cloudflare (แนะนำให้ใช้)
from IPython.display import clear_output

%cd /content
!wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
%cd $path
from google.colab import output
import subprocess
import threading
import time
import socket
import urllib.request
output.clear()

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\033[92m" + "ComfyUI เริ่มโปรแกรมเรียบร้อยแล้ว กำลังจะเปิด Cloudflared ให้ใช้..." + "\033[0m")

  p = subprocess.Popen(["cloudflared", "tunnel", "--loglevel", "info", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      clear_output()
      print("\033[92m" + "เสร็จสิ้น! ลิ้งค์อยู่ที่นี่นะ :", l[l.find("http"):], end='' + "\033[0m")

args = '--preview-method auto --dont-print-server' #@param {type: "string"}
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
!python main.py {args}